# Travel Advisor Code

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
from pprint import pprint
from res_config import api_key
import requests

In [5]:

#Travel Advisor
lat_url = "https://travel-advisor.p.rapidapi.com/restaurants/list-by-latlng"
locations = [[43.651070,-79.347015],[45.5019,-73.5674],[51.0447,-114.0719]]
#              ,[45.4215,-75.6972],[53.5461,-113.4937],[49.8954,-97.1385],
#              [43.5890,-79.6441],[49.2827,-123.1207],[43.7315,-79.7624],
#             [43.2557,-79.8711]]
#lat = 43.65107
#long = -79.347015

location_id = []
res_name = []
res_add = []
review_dates = []


for locs in locations:
    lat_querystring = {"latitude":locs[0],"longitude":locs[1],"limit":"30","currency":"CAD","distance":"2","lunit":"km","lang":"en_US"}

    headers = {
    "X-RapidAPI-Key": api_key,
    "X-RapidAPI-Host": "travel-advisor.p.rapidapi.com"
    }

    response = requests.get(lat_url, headers=headers, params=lat_querystring)



    for i in range(len(response.json()["data"])):
        location_id.append(response.json()["data"][i]["location_id"])

    loc_url = "https://travel-advisor.p.rapidapi.com/restaurants/get-details"

    for loc_id in location_id:
        loc_querystring = {"location_id":loc_id,"currency":"CAD","lang":"en_US"}

        resturants = requests.get(loc_url, headers=headers, params=loc_querystring)

        if "address" in resturants.json():
            res_name.append(resturants.json()["name"])
            res_add.append(resturants.json()["address"])

            if int(resturants.json()["num_reviews"]) > 0:
                review_dates.append(resturants.json()["reviews"][0]["published_date"])
            else:
                review_dates.append("")



data = {
    "Name" : res_name,
    "Address" : res_add,
    "Reviews" : review_dates
    
    
}


df = pd.DataFrame(data)

df_filtered = df[df['Reviews'] != ""]

df_filtered
df_filtered.to_csv("output/Travel_Advisor.csv")

,Name,Address,Reviews
0,Keating Channel Pub and Grill,"2 Villiers St, Toronto, Ontario M5A 1B1 Canada",2023-09-25T00:20:35-04:00
1,Sukhothai,"490 Front St E, Toronto, Ontario M5A 0J1 Canada",2023-06-12T20:29:25-04:00
2,The Aviary,"484A Front St E, Toronto, Ontario M5A 0J1 Canada",2022-08-14T07:22:13-04:00
4,District 28,"28 Logan Ave, Toronto, Ontario M4M 2M8 Canada",2018-02-05T08:21:55-05:00
5,Cherry Street Bar-B-Que,"275 Cherry St, Toronto, Ontario M5A 3L3 Canada",2023-09-03T08:19:17-04:00
...,...,...,...
172,Thai Express Restaurant Calgary,"751 3rd St SW The Core Shopping Centre, Calgar...",2020-03-14T08:45:18-04:00
173,Villa Madina,"751 3 St SW The CORE Calgary Eaton Centre, #F9...",2019-08-01T15:28:24-04:00
176,CRAFT Beer Market- Downtown Calgary,"345 10 Ave SW, Calgary, Alberta T2R 0A5 Canada",2023-07-28T16:29:42-04:00
178,Tim Hortons,"2525 Woodview Dr SW In Woodbine Square, Calgar...",2020-10-29T16:44:12-04:00
